<a href="https://colab.research.google.com/github/FlyAIBox/Agent_In_Action/blob/main/01-agent-llm-mcp/prompt-enginner/AgentLoopWithFunctionCalling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 具备函数调用机制的智能体

## 代码作用和目的

这个notebook演示了如何使用 **函数调用（Function Calling）** 技术来简化AI智能体的实现。相比传统的智能体实现方式，函数调用技术带来了以下重要优势：

### 🎯 核心目的
- **简化智能体循环**：通过函数调用技术，大幅减少解析LLM响应的复杂性
- **结构化响应**：模型原生支持结构化响应，确保返回有效的函数调用或标准文本回复
- **动态工具执行**：智能体可以根据需要动态调用不同的工具函数

### 🔧 技术特点
1. **无需自定义解析逻辑**：不需要设计严格的文本输出解析格式
2. **动态执行**：直接读取和执行模型返回的函数调用
3. **统一处理**：混合处理对话式回复和动作驱动的交互
4. **自动函数映射**：智能体自动将工具名称映射到对应的Python函数

### 📚 学习价值
- 理解函数调用在AI智能体中的应用
- 掌握如何设计工具函数和配置
- 学习智能体循环的基本实现模式
- 了解错误处理和异常管理机制

### 🚀 适用场景
- 文件操作和管理
- 数据查询和处理
- 自动化任务执行
- 智能助手开发

这个实现为初学者提供了一个清晰、易懂的智能体工具调用框架，是学习AI智能体开发的重要起点。


In [1]:
# 安装必要的依赖包
!!pip install openai==1.107.0

['Collecting openai==1.107.0',
 '  Downloading openai-1.107.0-py3-none-any.whl.metadata (29 kB)',
 'Requirement already satisfied: anyio<5,>=3.5.0 in /usr/local/lib/python3.12/dist-packages (from openai==1.107.0) (4.11.0)',
 'Requirement already satisfied: distro<2,>=1.7.0 in /usr/local/lib/python3.12/dist-packages (from openai==1.107.0) (1.9.0)',
 'Requirement already satisfied: httpx<1,>=0.23.0 in /usr/local/lib/python3.12/dist-packages (from openai==1.107.0) (0.28.1)',
 'Requirement already satisfied: jiter<1,>=0.4.0 in /usr/local/lib/python3.12/dist-packages (from openai==1.107.0) (0.11.1)',
 'Requirement already satisfied: pydantic<3,>=1.9.0 in /usr/local/lib/python3.12/dist-packages (from openai==1.107.0) (2.11.10)',
 'Requirement already satisfied: sniffio in /usr/local/lib/python3.12/dist-packages (from openai==1.107.0) (1.3.1)',
 'Requirement already satisfied: tqdm>4 in /usr/local/lib/python3.12/dist-packages (from openai==1.107.0) (4.67.1)',
 'Requirement already satisfied: 

In [2]:
# 导入必要的模块
import os, getpass

def _set_env(var: str):
    """
    设置环境变量的辅助函数

    参数:
        var (str): 要设置的环境变量名称

    功能:
        - 检查环境变量是否已存在
        - 如果不存在，则提示用户输入并设置
    """
    if not os.environ.get(var):  # 检查环境变量是否已设置
        os.environ[var] = getpass.getpass(f"{var}: ")  # 安全地获取用户输入

# 设置 OpenAI API 密钥
# 这是使用 OpenAI 模型所必需的
_set_env("OPENAI_API_KEY")
# 设置 OpenAI API代理地址 (例如：https://api.apiyi.com/v1）
_set_env("OPENAI_BASE_URL")

OPENAI_API_KEY: ··········
OPENAI_BASE_URL: ··········


In [4]:
# ===========================================
# 导入必要的库和模块
# ===========================================
import json  # 用于处理JSON数据格式
import os    # 用于操作系统相关操作（如文件列表、文件读取）
from typing import List  # 用于类型提示，提高代码可读性
from openai import OpenAI # 用于调用OpenAI API

# ===========================================
# 工具函数定义 - 智能体可以调用的具体功能
# ===========================================

def list_files() -> List[str]:
    """
    列出当前目录中的所有文件和文件夹

    返回值:
        List[str]: 包含当前目录中所有文件和文件夹名称的列表
    """
    return os.listdir(".")

def read_file(file_name: str) -> str:
    """
    读取指定文件的内容

    参数:
        file_name (str): 要读取的文件名

    返回值:
        str: 文件内容（如果成功）或错误信息（如果失败）
    """
    try:
        # 尝试以只读模式打开文件
        with open(file_name, "r") as file:
            return file.read()
    except FileNotFoundError:
        # 文件不存在时的错误处理
        return f"Error: {file_name} not found."
    except Exception as e:
        # 其他异常的处理（如权限问题、文件损坏等）
        return f"Error: {str(e)}"

def terminate(message: str) -> None:
    """
    终止智能体循环并显示总结消息

    参数:
        message (str): 要显示给用户的终止消息
    """
    print(f"Termination message: {message}")

# ===========================================
# 工具函数映射字典
# ===========================================
# 将工具名称映射到对应的Python函数
# 这样智能体就可以通过工具名称动态调用相应的函数
tool_functions = {
    "list_files": list_files,    # 文件列表工具
    "read_file": read_file,      # 文件读取工具
    "terminate": terminate       # 终止工具
}

# ===========================================
# 工具配置 - 定义智能体可以使用的工具
# ===========================================
# 这个配置告诉大语言模型有哪些工具可以使用
# 以及每个工具的参数要求
tools = [
    {
        "type": "function",  # 工具类型：函数调用
        "function": {
            "name": "list_files",  # 工具名称
            "description": "Returns a list of files in the directory.",  # 工具描述
            "parameters": {"type": "object", "properties": {}, "required": []}  # 参数配置（此工具无需参数）
        }
    },
    {
        "type": "function",
        "function": {
            "name": "read_file",
            "description": "Reads the content of a specified file in the directory.",
            "parameters": {
                "type": "object",
                "properties": {
                    "file_name": {"type": "string"}  # 定义参数类型
                },
                "required": ["file_name"]  # 必需参数列表
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "terminate",
            "description": "Terminates the conversation. No further actions or interactions are possible after this. Prints the provided message for the user.",
            "parameters": {
                "type": "object",
                "properties": {
                    "message": {"type": "string"},  # 终止消息参数
                },
                "required": ["message"]  # 消息参数是必需的
            }
        }
    }
]

# ===========================================
# 智能体规则和系统提示
# ===========================================
# 定义智能体的行为规则和角色定位
agent_rules = [{
    "role": "system",  # 系统角色，定义智能体的基本行为
    "content": """
You are an AI agent that can perform tasks by using available tools.

If a user asks about files, documents, or content, first list the files before reading them.

When you are done, terminate the conversation by using the "terminate" tool and I will provide the results to the user.
"""
}]

# ===========================================
# 智能体循环参数初始化
# ===========================================
iterations = 0        # 当前迭代次数计数器
max_iterations = 10   # 最大迭代次数限制，防止无限循环

# 获取用户任务输入
user_task = input("What would you like me to do? ")

# 初始化对话记忆，存储用户的任务
memory = [{"role": "user", "content": user_task}]

# ===========================================
# 智能体主循环 - 核心执行逻辑
# ===========================================
while iterations < max_iterations:

    # 构建完整的消息列表：系统规则 + 对话记忆
    messages = agent_rules + memory

    # 调用大语言模型，获取响应
    # 模型会根据用户任务和可用工具决定下一步行动
    client=OpenAI(
        base_url=os.environ['OPENAI_BASE_URL'],
        api_key=os.environ['OPENAI_API_KEY']
    )
    response = client.chat.completions.create(
        model="gpt-4o",  # 指定使用的模型
        messages=messages,      # 发送消息历史
        tools=tools,           # 提供可用工具列表
        max_tokens=1024       # 限制响应长度
    )


    # ===========================================
    # 处理工具调用响应
    # ===========================================
    if response.choices[0].message.tool_calls:
        # 如果模型决定调用工具，提取工具信息
        tool = response.choices[0].message.tool_calls[0]
        tool_name = tool.function.name                    # 获取工具名称
        tool_args = json.loads(tool.function.arguments)  # 解析工具参数

        # 构建动作记录，用于记忆管理
        action = {
            "tool_name": tool_name,
            "args": tool_args
        }

        # ===========================================
        # 工具执行逻辑
        # ===========================================
        if tool_name == "terminate":
            # 如果是终止工具，显示消息并退出循环
            print(f"Termination message: {tool_args['message']}")
            break
        elif tool_name in tool_functions:
            # 如果是已知工具，尝试执行
            try:
                # 动态调用对应的工具函数
                result = {"result": tool_functions[tool_name](**tool_args)}
            except Exception as e:
                # 工具执行出错时的错误处理
                result = {"error":f"Error executing {tool_name}: {str(e)}"}
        else:
            # 未知工具的错误处理
            result = {"error": f"Unknown tool: {tool_name}"}

        # 显示执行过程和结果
        print(f"Executing: {tool_name} with args {tool_args}")
        print(f"Result: {result}")

        # 将执行结果添加到对话记忆中
        # 这样模型就能记住之前的操作和结果
        memory.extend([
            {"role": "assistant", "content": json.dumps(action)},  # 记录智能体的动作
            {"role": "user", "content": json.dumps(result)}       # 记录执行结果
        ])
    else:
        # 如果模型没有调用工具，直接返回文本响应
        result = response.choices[0].message.content
        print(f"Response: {result}")
        break  # 结束循环


What would you like me to do? 告诉我当前目录里有什么内容
Executing: list_files with args {}
Result: {'result': ['.config', 'sample_data']}
Termination message: 该目录包含两个文件或文件夹：.config 和 sample_data。


# 📚 关键概念解析和学习要点

## 🔍 函数调用（Function Calling）的核心优势

### 1. **结构化响应处理**
```python
# 传统方式：需要解析非结构化文本
# "请执行 list_files 命令"
# 需要复杂的文本解析和命令提取逻辑

# 函数调用方式：直接获得结构化数据
tool_calls = response.choices[0].message.tool_calls
tool_name = tool.function.name  # 直接获取工具名称
tool_args = json.loads(tool.function.arguments)  # 直接获取参数
```

### 2. **动态工具执行**
```python
# 通过字典映射实现动态函数调用
tool_functions = {
    "list_files": list_files,
    "read_file": read_file,
    "terminate": terminate
}

# 根据模型返回的工具名称动态执行
result = tool_functions[tool_name](**tool_args)
```

## 🛠️ 智能体循环的核心组件

### 1. **工具定义**
- **函数实现**：具体的功能代码
- **工具配置**：告诉模型工具的存在和参数要求
- **映射字典**：连接工具名称和实际函数

### 2. **记忆管理**
```python
# 对话记忆包含完整的交互历史
memory = [
    {"role": "user", "content": "用户任务"},
    {"role": "assistant", "content": "智能体动作"},
    {"role": "user", "content": "执行结果"}
]
```

### 3. **错误处理**
- **JSON解析错误**：处理模型返回的格式错误
- **函数执行错误**：处理工具运行时的异常
- **未知工具错误**：处理模型调用了不存在的工具

## 🎯 学习重点

### 初学者应该关注：
1. **工具设计模式**：如何设计可复用的工具函数
2. **参数验证**：确保工具接收正确的参数
3. **错误处理**：优雅地处理各种异常情况
4. **记忆管理**：如何维护对话上下文
5. **循环控制**：防止无限循环的安全机制

### 进阶概念：
1. **工具链**：多个工具的组合使用
2. **条件执行**：根据结果决定下一步行动
3. **并行执行**：同时执行多个工具
4. **工具注册**：动态添加和移除工具

## ⚠️ 常见问题和解决方案

### 1. **JSON解析错误**
```python
try:
    tool_args = json.loads(tool.function.arguments)
except json.JSONDecodeError:
    # 处理格式错误，可以重试或使用默认值
```

### 2. **工具执行失败**
```python
try:
    result = tool_functions[tool_name](**tool_args)
except Exception as e:
    result = {"error": f"Error executing {tool_name}: {str(e)}"}
```

### 3. **无限循环**
- 设置最大迭代次数限制
- 使用终止工具明确结束对话
- 监控循环状态和资源使用

这个实现为智能体开发提供了一个坚实的基础，理解了这些概念后，你就可以构建更复杂的智能体系统了！
